In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from random import randint

image_files = ['circle.jpg', 'concentric_circles.jpg', 'parliament_clock.jpg']
inlier_thresholds = [600, 400, 600]

def makeCircle(img, points):
    p1 = points[0]
    p2 = points[1]
    p3 = points[2]

    if p1 == p2 or p2 == p3 or p1 == p3: # rmeove duplicates 
        return -1
    
    # math adapted from https://www.geeksforgeeks.org/equation-of-circle-when-three-points-on-the-circle-are-given/
    x_a = p2[1] - p1[1]
    x_b = p3[1] - p2[1]

    y_a = p2[0] - p1[0]
    y_b = p3[0] - p2[0]

    try: 
        slope_a = y_a / x_a;
        slope_b = y_b / x_b;
        
        if(abs(abs(slope_a) - abs(slope_b)) == 0): # collinear
            return -1
    
        center_x = int((slope_a * slope_a * (p1[0] - p3[0]) + slope_b * (p1[0] + p2[0]) - slope_a * (p2[0] + p3[0])) / (2 * (slope_b - slope_a)))
        center_y = int(-1 * (center_x - (p1[1] + p2[1]) / 2) / slope_a + (p1[1] + p2[1]) / 2)

        center_r = sqrt( (center_x - p1[1])**2 + (center_y - p1[0])**2 )
        if (center_r > img.shape[0]/2 or center_r > img.shape[1]/2): # exclude big circles
            return -1
        
        return [center_x, center_y, center_r]
    except: # sometimes divide by 0 error 
        return -1

def findInliers(img, circle, t): 
    inliers = []
    for i in range(0, img.shape[0]):
        for j in range(0, img.shape[1]):
            if img[i,j] == 255:
                
                center = [circle[0], circle[1]]
                x = j - center[0]
                y = i - center[1]

                euclidian_dist = sqrt(x**2 + y**2)
                distance = abs(euclidian_dist - circle[1])
                
                if distance < t: 
                    inliers.append((y, x))

    return inliers

def postProcessing(img, circle):
    inliers = findInliers(img, circle, post_process_threshold)
    print("post processed inliers %d" % len(inliers))
    
    x_coords = [p[0] for p in inliers]
    y_coords = [p[1] for p in inliers]
    _len = len(inliers)
    centroid = [int(- sum(x_coords) / _len), int(sum(y_coords) / _len)]
    
    radii = []
    for i in inliers:
        r = sqrt( (centroid[1] - i[1])**2 + (centroid[0] - i[0])**2 )
        radii.append(r)
    
    radius = np.mean(radii, axis=0)
    return [centroid[0], centroid[1], radius]

def overlayCircle(img, circle):
    return cv2.circle(img, (circle[0], circle[1]), int(circle[2]), (255, 255, 255), 2)

def RANSAC(fileName):
    src = cv2.imread(fileName, 0)
    blurred_img = cv2.GaussianBlur(src, (5, 5), cv2.BORDER_DEFAULT)
    binary_image = cv2.Canny(blurred_img, upper, lower)
    print("running RANSAC on: " + fileName)
    
    iterations = 0
    curr_inliers = -1
    best_circle = [0,0,0]
    inliers = []

    while curr_inliers < num_inliers:

        cur_points = []
        for i in range(3):
            point = (randint(0, binary_image.shape[0]-1), randint(0, binary_image.shape[1] - 1))
            while (binary_image[point[0], point[1]] == 0):
                point = (randint(0, binary_image.shape[0]-1), randint(0, binary_image.shape[1] - 1))
            cur_points.append(point)

        cur_circle = makeCircle(binary_image, cur_points)
        
        if not isinstance(cur_circle, list):
            continue

        inliers = findInliers(binary_image, cur_circle, normal_threshold)

        if len(inliers) > curr_inliers:
            curr_inliers = len(inliers)
            best_circle = cur_circle

            img = binary_image.copy()
            img = overlayCircle(img, best_circle)
            print("better fit found, iteration %d, inliers %d" %(iterations, len(inliers)))
            # draw current cirlce with circle drawing fn
            cv2.destroyWindow("current RANSAC best fit")
            cv2.imshow("current RANSAC best fit", img)
            cv2.waitKey(1)
        iterations += 1

    print("best result, iterations %d, inliers %d" %(iterations, len(inliers)))
    post_processed_circle = postProcessing(binary_image, best_circle)
    print("best circle (center, radius): " + str(best_circle))
    print("post processing circle (center, radius): " + str(post_processed_circle))

    post_processed = overlayCircle(binary_image, post_processed_circle)
    cv2.imshow("post processed image", post_processed)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

for i in range(len(image_files)):
    img = image_files[i]
    src = cv2.imread(img, 0)
    lower = 20
    upper = 100
    num_inliers = inlier_thresholds[i]
    normal_threshold = 3
    post_process_threshold = 2
    RANSAC(img)


running RANSAC on: circle.jpg
better fit found, iteration 0, inliers 28
better fit found, iteration 1, inliers 33
better fit found, iteration 2, inliers 199
better fit found, iteration 15, inliers 326
better fit found, iteration 32, inliers 393
better fit found, iteration 48, inliers 522
better fit found, iteration 49, inliers 586
better fit found, iteration 61, inliers 653
